<a href="https://colab.research.google.com/github/bshongwe/Other-FCC-Solutions/blob/master/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:

# Explore the dataset
print("Dataset shape:", dataset.shape)
print("\nDataset info:")
print(dataset.info())
print("\nFirst few rows:")
print(dataset.head())
print("\nDataset description:")
print(dataset.describe())

# Check for categorical columns
print("\nCategorical columns:")
categorical_cols = dataset.select_dtypes(include=['object']).columns
print(categorical_cols.tolist())

# Convert categorical data to numerical data
dataset_processed = dataset.copy()

# Convert 'sex' column: male=1, female=0
dataset_processed['sex'] = dataset_processed['sex'].map({'male': 1, 'female': 0})

# Convert 'smoker' column: yes=1, no=0
dataset_processed['smoker'] = dataset_processed['smoker'].map({'yes': 1, 'no': 0})

# Convert 'region' column using one-hot encoding
region_dummies = pd.get_dummies(dataset_processed['region'], prefix='region')
dataset_processed = pd.concat([dataset_processed, region_dummies], axis=1)
dataset_processed = dataset_processed.drop('region', axis=1)

print("\nProcessed dataset shape:", dataset_processed.shape)
print("Processed dataset columns:", dataset_processed.columns.tolist())

# Split the data into training and testing sets (80% train, 20% test)
train_dataset = dataset_processed.sample(frac=0.8, random_state=0)
test_dataset = dataset_processed.drop(train_dataset.index)

print(f"\nTrain dataset shape: {train_dataset.shape}")
print(f"Test dataset shape: {test_dataset.shape}")

# Separate features and labels
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

print(f"\nTrain features shape: {train_dataset.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Test features shape: {test_dataset.shape}")
print(f"Test labels shape: {test_labels.shape}")

# Normalize the features
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

# Build the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Compile the model
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.RMSprop(0.001),
              metrics=['mae', 'mse'])

# Display model architecture
model.summary()

# Train the model
EPOCHS = 500

history = model.fit(
    normed_train_data, train_labels,
    epochs=EPOCHS, validation_split=0.2, verbose=0,
    callbacks=[tfdocs.modeling.EpochDots()])

# Plot training history
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error')
plt.plot(hist['epoch'], hist['mae'], label='Train Error')
plt.plot(hist['epoch'], hist['val_mae'], label='Val Error')
plt.legend()

plt.subplot(1, 2, 2)
plt.xlabel('Epoch')
plt.ylabel('Mean Square Error')
plt.plot(hist['epoch'], hist['mse'], label='Train Error')
plt.plot(hist['epoch'], hist['val_mse'], label='Val Error')
plt.legend()

plt.tight_layout()
plt.show()

# Update the datasets for the final test (use normalized data)
train_dataset = normed_train_data
test_dataset = normed_test_data

print("Model training completed!")
print(f"Final training MAE: {hist['mae'].iloc[-1]:.2f}")
print(f"Final validation MAE: {hist['val_mae'].iloc[-1]:.2f}")

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
